In [ ]:
# @title HW 1 
# @markdown Student Info
# @markdown Name: [Skyler Thomas]
# @markdown JHED-ID: [sthom215]

# @markdown Load up the calcium image video in the file: TEST MOVIE 00001-small-motion.tif. This file represents the raw fluorescence video in TIF format, common for calcium imaging.

import os
from pathlib import Path
import numpy as np
import plotly.express as px
home = Path.cwd()

## Part A 
Write a script to play the data as a video (for Python users, look up the package plotly). Observe the changes in the video as a function of time and how the neurons “wiggle.

In [ ]:
from sthom215_problem_1_ex_1 import TIFF_IO

tiff1 = 'TEST_MOVIE_00001-small-motion.tif'

tiffreader = TIFF_IO(home / tiff1)
movie = tiffreader.read_tiff()

In [ ]:
# @title View a frame
# plt.figure(figsize=(4,4), tight_layout=True)
data = movie[301:400, :,:]
px.imshow(data, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice")) # @param i {type:"integer"}
# plt.axis('off')

## Part B
Pick two frames that seem to be particularly offset from each other. Compute their correlation, and repeat for different shifts in space.

In [ ]:
# @title load tiff file using the tifffile module
import tifffile

dir = os.listdir()
tiff1 = 'TEST_MOVIE_00001-small-motion.tif'

if tiff1 not in dir:
  cwd = home /'drive/MyDrive/NSC_HW1'
  movie = tifffile.imread(cwd/tiff1)
else:
  movie = tifffile.imread(home/tiff1)

In [ ]:
# @title Choose two frames to get peak correlation:
from sthom215_problem_1_ex_2 import XCORR
frame1 = 32 # @param {type:"slider", min:0, max:100, step:1}
frame2 = 52 # @param {type:"slider", min:0, max:100, step:1}
xcor = XCORR(movie[frame1,:,:], movie[frame2,:,:])
tmp = xcor.get_xcorr_fft()

print(f'max_xcorr is: {tmp[0]}')

In [ ]:
xcor.plot_xcorr_surface_3d()

In [ ]:
xcor.plot_xcorr_heatmap()

# Problem 2: Summary images. 

Load up the calcium image video in the file: $\texttt{TEST MOVIE 00001-small.tif}$. This file represents the raw fluorescence video in TIF format, but without the motion. One of the most common ways that people analyze functional data is to try to identify individual cells in the data in a “summary image”. A summary image condenses an entire video sequence into a single image, wherein each pixel represents a summary of the entire time-trace at the location of that pixel in the video.

## Part A 

Plot the “mean”, “median”, and “variance” images for the data provided. 

In [ ]:
# @title define functions
# Plot the “mean”, “median”, and “variance” images for the data provided. 
from sthom215_problem_2_ex_1 import SummaryImages

fname = 'TEST_MOVIE_00001-small-motion.tif'

summaryimages1 = SummaryImages(home/fname, fnorm=False)
summaryimages2 = SummaryImages(home/fname, fnorm=True)

In [ ]:
summaryimages1.plot_mean_median_variance()

In [ ]:
summaryimages2.plot_mean_median_variance()

### What do you notice about the visible cells in each of these?

The cells in the mean and median image cells look similar. This suggests that the median and mean pixels are near each other. In the variance image, no cells are visible. This suggests that after mean centering, the majority of the pixels were set to zero while only the intense pixels remained. Additionally, the variance may not all be due to cells in the image. Some of the bright spots might be artifacts from how the video was recorded or large changes in pixels due to shifts across frames. 

# Problem 2: Summary images. 

Load up the calcium image video in the file: $\texttt{TEST MOVIE 00001-small.tif}$. This file represents the raw fluorescence video in TIF format, but without the motion. One of the most common ways that people analyze functional data is to try to identify individual cells in the data in a “summary image”. A summary image condenses an entire video sequence into a single image, wherein each pixel represents a summary of the entire time-trace at the location of that pixel in the video.

## Part B


* ### What would you expect a good statistic for a summary image to capture?


A good statistic for a summary image should provide an overview of the content across all of the frames in movies across time. So I should have an idea of not only color but of shape and even changes in hue.


* ### What other statistics would you think could work? 


An image that summarizes the average shape across all images could be a summary. An image that summarizes the average edges across all images could also work. A plot that shows the average distribution across pixels.

In [ ]:
from sthom215_problem_1_ex_1 import TIFF_IO

tiff1 = 'TEST_MOVIE_00001-small-motion.tif'

tiffreader = TIFF_IO(home / tiff1)
movie = tiffreader.read_tiff()

In [ ]:
# @title Get pixel distribution at a single pixel across the entire movie
_, height, width = movie.shape

x_coord = 0 # @param {type:"slider", min:0, max:width, step:1}
y_coord = 0 # @param {type:"slider", min:0, max:height, step:1}

from sthom215_problem_2_ex_2 import get_pixelDistribution
#sigma controls smoothing of the distribution
get_pixelDistribution(movie, 2 ,1, sigma=15) 

In [ ]:
# @title Edge detection summary
from sthom215_problem_2_ex_2 import get_edge_mean
import matplotlib.pyplot as plt

plt.imshow(get_edge_mean(movie), cmap='gray')
plt.title("Average Edge Intensity of Cellular Activity Over Time")
plt.axis('off')
plt.tight_layout()

### Were you right? 

I think my pixel approach is interesting but the scale is too small to be useful across an entire movie. It is better suited to studying the statistics of an image at a single frame. 

I think my edge detetction approach is a decent summary because it gives the viewer an idea of active areas of change across all of the frames in the movie. 


# Problem 3: Drawing Regions Of Interest (ROIs) 

Given the summary images, the next step is to draw ROIs around the image to isolate the pixels belonging to a specific cell.

## Part A: 

* Write a short script that takes in a summary image and potentially other information (i.e., a seed pixel selected by the user), and outputs a binary mask over an ROI. 

* Find 5 different ROIs in the data provided.

In [ ]:
# @markdown Load the video file using the provided TIFF_IO class.
from sthom215_problem_2_ex_1 import SummaryImages
from sthom215_problem_3_ex_1 import get_roi_2, get_bounding_boxes
fname = 'TEST_MOVIE_00001-small.tif'
summaryimage = SummaryImages(home/fname)
summaryimage = summaryimage.get_mean()
sigma = 3; kernel = 7
ROIs = get_roi_2(summaryimage, sigma, kernel)
rois_bounding_boxes = get_bounding_boxes(ROIs, True)  

# Problem 3: Drawing Regions Of Interest (ROIs) 

Given the summary images, the next step is to draw ROIs around the image to isolate the pixels belonging to a specific cell.

## Part B: 

* How might you assess how good your script did?

I can assess how well my script did by comparing my ROI to the raw image to see how well it captures the details or separates the ROI from the rest of the image frame. This visual inspection would cary in its accuracy betwen me an someone who is an expert at determining what is a neuron in an image and someone who is novice like myself. So, I could also have other people come and visually inspect the output of the script to determine whether or not the script did a good job.

This could also be done staistically by looking at the how much overlap there is between the ROI in the mask and the original image. 

# Problem 4: Time-trace estimation

With the ROIs, time-traces need to be extracted by identifying the relative brightness of that region
of interest in each frame

## Part A:

* Write a function that identifies, for the ROIs identified in Problem 2, their time-traces.

* Describe your approach.

In [ ]:
# @markdown Load the video file using the provided TIFF_IO class.
from sthom215_problem_1_ex_1 import TIFF_IO

fname = 'TEST_MOVIE_00001-small-motion.tif'
tiffreader = TIFF_IO(home/fname)
movie = tiffreader.read_tiff()
print("movie has shape:", movie.shape)

In [ ]:
from glob import glob
from sthom215_problem_3_ex_1 import get_hw3_roi #get rois from exercise 3
from sthom215_problem_4_ex_1 import create_mask_for_bounding_box, ROI
import plotly.express as px
box,mask = create_mask_for_bounding_box(movie, ROI[3])
px.imshow(box[0:100], animation_frame=0)

In [ ]:
from glob import glob
from sthom215_problem_3_ex_1 import get_hw3_roi #get rois from exercise 3
from sthom215_problem_4_ex_1 import create_mask_for_bounding_box, ROI
import plotly.express as px
box,mask = create_mask_for_bounding_box(movie, ROI[6])
px.imshow(box[0:100], animation_frame=0)

In [ ]:
from glob import glob
from sthom215_problem_3_ex_1 import get_hw3_roi #get rois from exercise 3
from sthom215_problem_4_ex_1 import create_mask_for_bounding_box, ROI
import plotly.express as px
box,mask = create_mask_for_bounding_box(movie, ROI[13])
px.imshow(box[0:100], animation_frame=0)

In [ ]:
from glob import glob
from sthom215_problem_3_ex_1 import get_hw3_roi #get rois from exercise 3
from sthom215_problem_4_ex_1 import create_mask_for_bounding_box, ROI
import plotly.express as px
box,mask = create_mask_for_bounding_box(movie, ROI[1])
px.imshow(box[0:100], animation_frame=0)

In [ ]:
from glob import glob
from sthom215_problem_3_ex_1 import get_hw3_roi #get rois from exercise 3
from sthom215_problem_4_ex_1 import create_mask_for_bounding_box, ROI
import plotly.express as px
box,mask = create_mask_for_bounding_box(movie, ROI[7])
px.imshow(box[0:100], animation_frame=0)

# Problem 4: Time-trace estimation

With the ROIs, time-traces need to be extracted by identifying the relative brightness of that region
of interest in each frame

## Part B: 

* Do you think that the time-traces reflect the actual activity in the video? How might you test if they are accurate?


I think that the time traces reflect what is actually  happening in the video to the extent that the ROI in the video is accurate. The less accurate the ROI, the worse the representation of the time traces is.


Similar to checking how a single frame has an accurate ROI, we can employ experts to check the time traces or we can have many people check them and compare their output. These have the advantage that they could allow for fine tuning but both are expensive time wise with the former being just expensive in general. We can again use statistical methods as well, but errors in some of the frames could propagate to other frames as well, worsening the time traces.

# Problem 5: Matrix factorization 
The above procedure is common in many labs, however can cause errors due to the dependency on
the summary image and the multiple stages of processing. Matrix factorization has emerged as an
alternative approach for identifying ROIs from the full spatio-temporal video. Here we will explore
three different types of factorization and compare the results using a table.

## Part A

Run PCA on the pixels-by-time matrix obtained by vectorizing each frame into a column
vector.

In [ ]:
# @markdown Load the video file using the provided TIFF_IO class.
from sthom215_problem_1_ex_1 import TIFF_IO
import plotly.express as px
fname = 'TEST_MOVIE_00001-small-motion.tif'
tiffreader = TIFF_IO(home/fname)
movie = tiffreader.read_tiff()

In [ ]:
from sthom215_problem_5_ex_1 import get_hw5_1_solution
sol1 = get_hw5_1_solution(movie, num_components=2)
px.imshow(sol1, animation_frame=0, binary_string=False)

In [ ]:
from sthom215_problem_5_ex_1 import  PCA

pca1 = PCA(2)
trans1 = pca1.fit_transform(movie)
px.imshow(trans1, animation_frame=0, binary_string=False)


In [ ]:
pca1.skree_plot()

### How does the result change with the number of principal components you choose?

With variance explained by the top 20 eigenvectors and the top 3, there is very litte if any change the pixels of reconstructed plot. Ineed, looking at the screeplot shows that the first 5 eigenvalues, and therefore the top 5 eigenvectors, dominate over the following eigenvalues. This means that there should be a change in the reconstructions betwen using the top 3 and 1 eigenvectors, but not really much after that. In the context of the movie, this means that the tiff can be condensned into approximate a 1-5 images that summarize the variance across the pixels for the entire movie. 

Running the last cold block, one can see that 8 and 3 look similar and as we plot higher number components, the detail represented in them becomes finer and finer. 

In [ ]:
from sthom215_problem_5_ex_1 import get_hw5_1_solution
sol1 = get_hw5_1_solution(movie, num_components=10)
px.imshow(sol1, animation_frame=0, binary_string=False)

# Problem 5: Matrix factorization 
The above procedure is common in many labs, however can cause errors due to the dependency on
the summary image and the multiple stages of processing. Matrix factorization has emerged as an
alternative approach for identifying ROIs from the full spatio-temporal video. Here we will explore
three different types of factorization and compare the results using a table.

## Part B

RRun NMF on the pixels-by-time matrix as in part A, for a specific number of components
that you find reasonable.

In [ ]:
# @markdown Load the video file using the provided TIFF_IO class.
from sthom215_problem_1_ex_1 import TIFF_IO

fname = 'TEST_MOVIE_00001-small-motion.tif'
tiffreader = TIFF_IO(home/fname)
movie = tiffreader.read_tiff()

In [ ]:
from sthom215_problem_5_ex_2 import get_hw5_2_solution
import plotly_express as px
nmf_solution = get_hw5_2_solution(movie, 20)
px.imshow(nmf_solution, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))

In [ ]:
from sthom215_problem_5_ex_2 import get_hw5_2_solution
import plotly_express as px
nmf_solution = get_hw5_2_solution(movie, 10)
px.imshow(nmf_solution, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))

In [ ]:
from sthom215_problem_5_ex_2 import get_hw5_2_solution
import plotly_express as px
nmf_solution = get_hw5_2_solution(movie, 5)
px.imshow(nmf_solution, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))

In [ ]:
from sthom215_problem_5_ex_2 import get_hw5_2_solution
import plotly_express as px
nmf_solution = get_hw5_2_solution(movie, 3)
px.imshow(nmf_solution, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))

### What are the differences that you note? Is there a similar dependency on the rank of the decomposition?

I find that as the number of components of the factorization increases, the more each factor speacilizes in a single spatial set of pixels. The difference on the rank of the composition does not compress the pixels across time onto a single frame like with PCA. However, it does cause each image to finder a more granual neural activation pattern in each of the slices. 



# Problem 5: Matrix factorization 
The above procedure is common in many labs, however can cause errors due to the dependency on
the summary image and the multiple stages of processing. Matrix factorization has emerged as an
alternative approach for identifying ROIs from the full spatio-temporal video. Here we will explore
three different types of factorization and compare the results using a table.

## Part B

Now try ICA for the same pixels-by-time matrix as in part A (for a specific number of
components you find reasonable).

In [ ]:
# @markdown Load the video file using the provided TIFF_IO class.
from sthom215_problem_1_ex_1 import TIFF_IO

fname = 'TEST_MOVIE_00001-small-motion.tif'
tiffreader = TIFF_IO(home/fname)
movie = tiffreader.read_tiff()
print("movie has shape:", movie.shape)

In [ ]:
import plotly.express as px
from sthom215_problem_5_ex_3 import get_hw5_3_solution
num_components = 4
component_frames = get_hw5_3_solution(movie, num_components)
px.imshow(component_frames, animation_frame=0, binary_string=False)

### What are the differences that you note?

Each of the frames have both different colors a well as different promient featuers. This refelcts ICA's focus on finding statistical indepdence between signals. Each frame represents a different component that the algorithm fit with the its gaussian independence assumption. With four components, one component focuses on vascularity in the frames while another looks liek it focuses on the nuclei in the frames across different cells. 
